In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [12]:
def read_and_find_features(image_path, scale_percent=30):
    """
    Read an image from the given path, resize it by a percentage, and detect keypoints and descriptors using ORB.

    Args:
    - image_path (str): The path to the image file.
    - scale_percent (int): The percent (relative to the original image size) to which the image should be scaled.

    Returns:
    - keypoints (list of cv2.KeyPoint): The detected keypoints in the image.
    - descriptors (np.ndarray): The feature descriptors corresponding to the keypoints.
    - resized_image (np.ndarray): The resized image read from the file in BGR format.
    """
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be read.")

    # Calculate the new dimensions
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)

    # Resize the image
    resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

    # Initialize ORB detector
    orb = cv2.ORB_create()

    # Find the keypoints and descriptors with ORB
    keypoints, descriptors = orb.detectAndCompute(resized_image, None)
    
    return keypoints, descriptors, resized_image


def match_features(descriptors1, descriptors2):
    """
    Match feature descriptors between two sets of descriptors using the BFMatcher with NORM_HAMMING.

    Args:
    - descriptors1 (np.ndarray): Feature descriptors of the first image.
    - descriptors2 (np.ndarray): Feature descriptors of the second image.

    Returns:
    - matches (list of cv2.DMatch): The best matches found between the two descriptor sets.
    
    The function initializes a BFMatcher object with the NORM_HAMMING metric, suitable for ORB features,
    and uses it to find the best matches between two descriptor sets.
    """
    # Create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    # Match descriptors
    matches = bf.match(descriptors1, descriptors2)
    
    # Sort them in the order of their distance (the lower the better)
    matches = sorted(matches, key=lambda x: x.distance)
    
    return matches

def stitch_images(image1, keypoints1, image2, keypoints2, matches):
    """
    Stitch two images into a panorama using the matches of keypoints between them.

    Args:
    - image1 (np.ndarray): The first image to stitch.
    - keypoints1 (list of cv2.KeyPoint): Keypoints of the first image.
    - image2 (np.ndarray): The second image to stitch.
    - keypoints2 (list of cv2.KeyPoint): Keypoints of the second image.
    - matches (list of cv2.DMatch): Matches between the keypoints of the two images.

    Returns:
    - panorama (np.ndarray): The stitched panorama image.
    
    This function computes the homography matrix using the keypoints of the matches found. It then warps the
    perspective of the second image to align with the first image and stitches them together. The resulting image is
    the panorama of the two images.
    """
    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    try:
        h, mask = cv2.findHomography(points2, points1, cv2.RANSAC)
    except cv2.error as e:
        print(f"Error computing homography: {e}")
        return None


    # Use homography
    height, width, channels = image1.shape
    panorama = cv2.warpPerspective(image2, h, (width * 2, height))
    
    # Stitch the images together
    panorama[0:image1.shape[0], 0:image1.shape[1]] = image1

    return panorama

def create_panorama(image_left, image_mid, image_right, keypoints_left, keypoints_mid, keypoints_right, descriptors_left, descriptors_mid, descriptors_right):
    """
    Create a panorama with the middle image as the reference, stitching the left and right images to it.

    Args:
    - image_left (np.ndarray): The left image to be stitched.
    - image_mid (np.ndarray): The middle reference image.
    - image_right (np.ndarray): The right image to be stitched.
    - keypoints_left (list of cv2.KeyPoint): Keypoints of the left image.
    - keypoints_mid (list of cv2.KeyPoint): Keypoints of the middle image.
    - keypoints_right (list of cv2.KeyPoint): Keypoints of the right image.
    - descriptors_left (np.ndarray): Descriptors of the left image.
    - descriptors_mid (np.ndarray): Descriptors of the middle image.
    - descriptors_right (np.ndarray): Descriptors of the right image.

    Returns:
    - panorama (np.ndarray): The final stitched panorama image.
    
    The function first matches features between the middle and left images, and then between the middle and right images.
    It stitches the left image to the middle, then stitches the right image to the combined left and middle images, 
    all the while keeping the middle image as the central, unaltered reference.
    """
    # Match features between middle and left images, then stitch them
    matches_left_mid = match_features(descriptors_left, descriptors_mid)
    left_to_mid = stitch_images(image_left, keypoints_left, image_mid, keypoints_mid, matches_left_mid)

    # Match features between middle and right images, then stitch them
    matches_right_mid = match_features(descriptors_right, descriptors_mid)
    right_to_mid = stitch_images(image_mid, keypoints_mid, image_right, keypoints_right, matches_right_mid)

    # The right_to_mid image will have extra width to accommodate the warped right image
    # We need to crop this before combining with the left_to_mid image
    right_to_mid_cropped = right_to_mid[:, 0:image_mid.shape[1]]

    # Combine the left_to_mid and right_to_mid images to create the full panorama
    # Note that the dimensions of left_to_mid and right_to_mid_cropped must match exactly for this to work
    panorama = np.concatenate((left_to_mid, right_to_mid_cropped), axis=1)

    return panorama




In [13]:
# Nombres de archivo de las imágenes de entrada
filenames = ['IMG_IZQ.JPG', 'IMG_MID.JPG', 'IMG_DER.JPG']
H_filenames = ['H_IZQ.jpeg', 'H_MID.jpeg', 'H_DER.jpeg']
V_filenames = ['V_IZQ.jpeg', 'V_MID.jpeg', 'V_DER.jpeg']
profe_filenames = ['img1.png','img2.png','img3.png']

# Read images (replace paths with your actual image paths)
keypoints_left, descriptors_left, image_left = read_and_find_features(H_filenames[0])
keypoints_mid, descriptors_mid, image_mid = read_and_find_features(H_filenames[1])
keypoints_right, descriptors_right, image_right = read_and_find_features(H_filenames[2])

# Create the panorama
final_panorama = create_panorama(image_left, image_mid, image_right, keypoints_left, keypoints_mid, keypoints_right, descriptors_left, descriptors_mid, descriptors_right)

# Display the final panorama
cv2.imshow('Final Panorama', final_panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Save the panorama image if needed
cv2.imwrite('final_panorama.jpg', final_panorama)


IndexError: tuple index out of range